In [16]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy import SpotifyClientCredentials
from dotenv import load_dotenv
import json

In [142]:
spotifyClient = os.getenv('SPOTIFY_CLIENT')
spotifySecret = os.getenv('SPOTIFY_SECRET')
auth_manager = SpotifyClientCredentials(client_id=spotifyClient, client_secret=spotifySecret)

sp = spotipy.Spotify(
    auth_manager=auth_manager,
    requests_timeout=10,
    retries=10
)

In [143]:
result = sp.search(q='20 años', type='album', limit=1)

In [ ]:
album_items = result['albums']['items'][0]
album_id = album_items['id']
tracks_raw = sp.album_tracks(album_id=album_id)['items']

In [168]:
result

{'albums': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=20%20a%C3%B1os&type=album',
  'limit': 1,
  'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=20%20a%C3%B1os&type=album',
  'offset': 0,
  'previous': None,
  'total': 899,
  'items': [{'album_type': 'album',
    'total_tracks': 10,
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',
     'SK',
     'ES',
     'SE',
     'CH',
     'TW',
     'TR',
     'UY',
     'US',
     'GB',
     'AD',
     'LI',
     'MC',
     'ID',
   

In [158]:
track_dict = {
    'track_number': [],
    'name': [],
    'duration': [],
    'song_id': [],
}

for track in tracks_raw:
    track_dict['track_number'].append(track['track_number'])
    track_dict['name'].append(track['name'])
    track_dict['duration'].append(track['duration_ms'])
    track_dict['song_id'].append(track['id'])

df_raw = pd.DataFrame(track_dict)
df_raw

,track_number,name,duration,song_id
0,1,Entrégate,263906,5fj3zNkXfOlrJGVcZBId6D
1,2,Oro De Ley,238933,2PZtKB8fDoDGLot27oUnWH
2,3,Tengo Todo Excepto a Ti,272826,27XboT5Wb8VOn7A0heo3Ei
3,4,Será Que No Me Amas (Blame It on the Boogie),249000,0ULXIAoxSZBcOwygC3PA0i
4,5,Amante Del Amor,202200,3HE0wPD30AjKMKtxNaKFQ4
5,6,Hoy El Aire Huele a Ti,224493,2ZYvD9zQf1iBLelkgkUl7U
6,7,Cuestión De Piel,264573,1PdgPj8tdE2OAhH40J4f4g
7,8,Más Allá De Todo,245133,7CbgafOaqArpdlZVlTpNeA
8,9,Alguien Como Tú,259066,4syvq0aJLeWu0UA9VYcgob
9,10,Más,191826,1KRaDU2NWnUN28Dj1wlnl8
